In [1]:
import numpy as np
import yaml
import shutil
import uuid
from tensorflow.keras import backend as K
from nn_tools import read_count
import tensorflow as tf
from data_bank import data_selector
import model_builders as mb
import os
from os.path import join
import matplotlib.pyplot as plt
import sys
from adversarial_attacks.spsa import spsa, spsa_T1
from adversarial_attacks.df_attacks import attack_network
from tensorflow.keras.applications.resnet50 import preprocess_input as res_prep
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_prep
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
import pandas as pd

In [2]:
from Demo_evaluate_network import print_certainty_vs_distance_table
import math
from scipy.special import softmax
import seaborn as sns

# Iterate over models

In [3]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Set up computational resource 
os.environ["CUDA_VISIBLE_DEVICES"]= '2'

# Turn on soft memory allocation
tf_config = tf.compat.v1.ConfigProto()
tf_config.gpu_options.allow_growth = True
tf_config.log_device_placement = False
sess = tf.compat.v1.Session(config=tf_config)

In [4]:
evaluation = pd.DataFrame([])

In [5]:
with open('./config_files/create_images_shades.yml')as ymlfile:
    cgf1 = yaml.load(ymlfile, Loader =yaml.SafeLoader)


data_loader = data_selector('existing', cgf1['DATA']['arguments'])
data_test, labels_test, diff_test = data_loader.load_data()

In [ ]:
for filename in os.listdir('./model'):
    
    
    # Remove hidden files
    if filename.startswith('.'):
        break
    
    print('Checking model: ',filename)
    
    configfile = './model/'+ filename +'/config.yml'

    with open(configfile) as ymlfile:
        cgf = yaml.load(ymlfile, Loader =yaml.SafeLoader)
        
    data_loader = data_selector('existing', cgf['DATASET_TRAIN']['arguments'])
    
    data, labels, diff = data_loader.load_data() 
    
    
    # Get input and output shape
    input_shape = data.shape[1:]
    #output_shape = labels.shape[1];
    print('input_shape', input_shape)
    # Set the default precision 
    model_precision = cgf['MODEL_METADATA']['precision']
    K.set_floatx(model_precision)

    model_id = cgf['MODEL_METADATA']['model_number_arguments']['model_id']
    model_path = join('model', str(model_id))
    filepath = cgf['MODEL_METADATA']['save_best_model']['arguments']['filepath']

    #original_data = cgf['DATASET']['arguments']['original_images']
    #bels = cgf['DATASET']['arguments']['labels']

    weights_path = './model/' + filename + '/'+ filepath

    optimizer = cgf['TRAIN']['optim']['type']
    loss_type = cgf['TRAIN']['loss']['type']
    metric_list = list(cgf['TRAIN']['metrics'].values())

    if loss_type == 'SparseCategoricalCrossentropy':
        loss_type = SparseCategoricalCrossentropy(from_logits=False)
        metric_list = [SparseCategoricalAccuracy()]
        output_shape = 2
        labels = np.reshape(labels, (-1))

    model_name = cgf['MODEL']['name']
    model_arguments = cgf['MODEL']['arguments']
    #model = mb.model_selector(model_name, input_shape, output_shape, model_arguments)

    model = tf.keras.models.load_model(weights_path)

    # Preprocessing
    if model_name =='resnet':
        preprocessing = res_prep
        data = 255*data
        data = data - 122
        labels = np.reshape(labels,(-1))
        #data = tf.cast(data, dtype=tf.float32)
        #labels = tf.cast(data, dtype=tf.float32)
    elif model_name == 'vgg16':
        preprocessing = vgg_prep
        data = 255*data
        data = data - 122
        labels = np.reshape(labels,(-1))
        #data = tf.cast(data, dtype=tf.float32)
        #labels = tf.cast(data, dtype=tf.float32)
    else:
        preprocessing = None 
        data = 255*data
        data = data - 122
        data = np.expand_dims(data, axis = 3)

    model.compile(optimizer=optimizer,
                  loss=loss_type,
                  metrics = metric_list)

    model.trainable = False
    
    
    print("Evaluate on training data")
    results_train = model.evaluate(tf.convert_to_tensor(data), tf.convert_to_tensor(labels), batch_size=1)
    print("Loss :{}, Accuracy: {}%.".format(results_train[0], results_train[1]))
    
    print("Evaluate on testing data")
    results_test = model.evaluate(tf.convert_to_tensor(data_test), tf.convert_to_tensor(labels_test), batch_size=1)
    print("Loss :{}, Accuracy: {}%.".format(results_test[0], results_test[1]))
    
    evaluation_dictionary = {'model': model_name, 'model id': filename, 
                             'train data': cgf['DATASET_TRAIN']['arguments']['images'], 
                             'train accuracy': results_train[1], 'train loss': results_train[0],
                             'test data': cgf1['DATA']['arguments']['images'],
                             'test accuracy': results_test[1], 'test loss': results_test[0]}
    
    evaluation_of_model = pd.Series(data = evaluation_dictionary, index=['model', 'model id', 'train data',
                                                                        'train accuracy', 'train loss',
                                                                        'test data', 'test accuracy',
                                                                        'test loss'])
    evaluation = evaluation.append(evaluation_of_model, ignore_index = True)

Checking model:  800
input_shape (224, 224, 1)
Evaluate on training data
10000/10000 [==============================] - 52s 5ms/step - loss: 0.3566 - sparse_categorical_accuracy: 0.9827
Loss :0.35655948120778014, Accuracy: 0.9827%.
Evaluate on testing data
5000/5000 [==============================] - 27s 5ms/step - loss: 0.6931 - sparse_categorical_accuracy: 0.5052
Loss :0.6931471824645996, Accuracy: 0.5052%.
Checking model:  500
input_shape (224, 224, 1)
Evaluate on training data
10000/10000 [==============================] - 1738s 174ms/step - loss: 2.9148 - sparse_categorical_accuracy: 0.7856
Loss :2.914763532138698, Accuracy: 0.7856%.
Evaluate on testing data
5000/5000 [==============================] - 872s 174ms/step - loss: 4.6296 - sparse_categorical_accuracy: 0.5086
Loss :4.629649094330826, Accuracy: 0.5086%.
Checking model:  303
input_shape (224, 224, 1)
Evaluate on training data
10000/10000 [==============================] - 2110s 211ms/step - loss: 0.9639 - sparse_categoric

In [9]:
evaluation.to_pickle('evaluation_of_trained_models.pkl')